In [1]:
import pandas as pd

# query = "Query_NYT_5y_surveillance"
query = "Query_NYT_13y_surveillance"
# query = "Query_US_5y_Svalbard"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

df = pd.read_feather(FEATHER_FILE)
df = df.drop(df[df.text_downloaded == False].index)
df = df.reset_index(drop=True)

df.sample(3)

,stories_id,title,url,year,text,text_downloaded
263,1284995352,"The Tension Between America and Iran, Explained",https://www.nytimes.com/2019/05/16/world/middl...,2019,Supported by\nThe Tension Between America and ...,True
2184,2762647811,A Performance Review for President Biden,https://www.nytimes.com/2023/09/07/opinion/let...,2023,Supported by\nletters\nA Performance Review fo...,True
2838,1159575973,Why the N.S.A. Isn’t Howling Over Restrictions...,https://www.nytimes.com/2015/05/02/us/politics...,2015,WASHINGTON — For years after the attacks of Se...,True


In [2]:
import requests
import json
from tqdm.notebook import trange  # to display a progress bar
import ipywidgets as widgets

# Tests
# curl -X POST http://localhost:8000/predict.json -H "Content-Type: application/json" -d '{"text": "Federal agents show stronger force at Portland protests despite order to withdraw" }'
# curl -X POST https://nyt-news-labeler-qoqojvpyua-ez.a.run.app/predict.json -H "Content-Type: application/json" -d '{"text": "Federal agents show stronger force at Portland protests despite order to withdraw" }'

# URL = 'http://nyt-news-labeler:8000'
URL = "https://nyt-news-labeler-qoqojvpyua-ez.a.run.app/"

headers = {"Content-Type": "application/json"}

tags = [{}] * len(df)

for index in trange(0, len(df)):

    text = df.loc[index].text
    text = text.replace("\n", "")

    try:
        data = {"text": text}  # article's text

        response = requests.post(
            URL + "predict.json", data=json.dumps(data), headers=headers
        )

        _json = json.loads(response.text)

        tags[index] = _json["predictions"]['descriptors3000']

    except Exception as e:
        print(tags[index])

df["tags"] = tags


  0%|          | 0/3359 [00:00<?, ?it/s]

{}
{}


## Save Dataset

In [5]:
df = df.drop(df[df.tags == {}].index)

df = df.reset_index(drop=True)

df.to_feather(FEATHER_FILE)

In [8]:
df.sample(10)

,stories_id,title,url,year,text,text_downloaded,tags
2536,592164228,Letter: Wiretaps: Which Statement Is True?,http://www.nytimes.com/2017/03/10/opinion/wire...,2017,SKIP ADVERTISEMENT\nSupported by\nLetter\nWire...,True,[{'label': 'wiretapping and other eavesdroppin...
2890,1247337259,Inching Toward Eradication,http://www.nytimes.com/2014/10/26/arts/artsspe...,2014,Supported by\nInching Toward Eradication\nSinc...,True,"[{'label': 'shows (exhibits)', 'score': '0.308..."
172,1243155369,Justice Dept. Watchdog’s Review of Russia Inqu...,https://www.nytimes.com/2019/04/09/us/politics...,2019,Supported by\nJustice Dept. Watchdog’s Review ...,True,[{'label': 'wiretapping and other eavesdroppin...
2221,2783653030,An Invasive Mosquito Threatens Catastrophe in ...,https://www.nytimes.com/2023/09/29/health/mosq...,2023,An Invasive Mosquito Threatens Catastrophe in ...,True,"[{'label': 'pesticides', 'score': '0.91227'}, ..."
3218,1034540810,Justice Dept. Is Accused of Misleading Public ...,https://www.nytimes.com/2011/09/22/us/politics...,2011,Supported by\nPublic Said to Be Misled on Use ...,True,"[{'label': 'politics and government', 'score':..."
2707,1022357447,White Woman Calls the Police on a Black Child ...,https://www.nytimes.com/2018/10/12/nyregion/wo...,2018,"Supported by\nA White Woman, Teresa Klein, Cal...",True,"[{'label': 'blacks (in us)', 'score': '0.79486..."
1081,2334801352,"James Goldston, Former TV News Chief, Helps th...",https://www.nytimes.com/2022/06/10/business/me...,2022,Supported by\nWho Is the Former TV News Chief ...,True,"[{'label': 'television', 'score': '0.51923'}, ..."
1661,2572822491,"On Russian state TV, Biden’s visit is cast as ...",https://www.nytimes.com/live/2023/02/20/world/...,2023,"Amid Air Raid Sirens, Biden Makes Surprise Ukr...",True,"[{'label': 'international relations', 'score':..."
1553,2556937721,The Chinese spy balloon moved from Alaska to S...,https://www.nytimes.com/live/2023/02/04/us/chi...,2023,U.S. Shoots Down Surveillance BalloonChina Con...,True,"[{'label': 'armament, defense and military for..."
1707,2595409759,The U.S. Program That Brought H.I.V. Treatment...,https://www.nytimes.com/2023/03/14/health/pepf...,2023,Supported by\nGlobal Health\nThe U.S. Program ...,True,[{'label': 'acquired immune deficiency syndrom...
